In [64]:
import numpy as np
import matplotlib.pyplot as plt
from itertools import product
from scipy.optimize import minimize

In [65]:
class LinearMDP_train():
    def __init__(self, action_space, delta, xi_norm, seed=1):
        np.random.seed(seed)
        self.state_space = ['x1', 'x2', 'x3', 'x4', 'x5']
        self.action_space = action_space
        self.initial_state = 'x1'
        self.theta = [np.zeros(4),
                      np.array([0,0,0,1]),
                      np.array([0,0,0,1])]
        self.delta = delta
        self.H = 3
        Xi = np.full(len(self.action_space[0]), 1)
        self.Xi = xi_norm * Xi / np.linalg.norm(Xi, 1)

    def reset(self):
        self.S = [self.initial_state]
        self.A = [] # save the action history
        self.R = [] # save the reward history
        self.h = 0 # reset the step to 0
        self.feature = [] # save the feature trajectory
        self.feature_a = [] # save the full feature trajectory
        self.current_state = self.initial_state # reset the current state to initial state

    def phi(self, current_state, A):
        if current_state == 'x1':
            phi = np.array([1 - self.delta - self.Xi @ A, 0, 0, self.delta + self.Xi @ A])
        elif current_state == 'x2':
            phi = np.array([0, 1 - self.delta - self.Xi @ A, 0, self.delta + self.Xi @ A])
        elif current_state == 'x3':
            phi = np.array([0, 0, 1 - self.delta - self.Xi @ A, self.delta + self.Xi @ A])
        elif current_state == 'x4':
            phi = np.array([0, 0, 1, 0])
        else:
            phi = np.array([0, 0, 0, 1])
        return phi

    def add_state(self, s):
        self.S.append(s)
        self.current_state = s

    def update_state(self, phi, h):
        # calculate the transition probability
        if h == 0:
            prob = [phi[0] * (1 - 0.001), 0, phi[0] * 0.001, phi[3]]
        elif h == 1:
            if phi[2] == 1:
                prob = [0, 0, 1, 0]
            elif phi[3] == 1:
                prob = [0, 0, 0, 1]
            else:
                prob = [0, phi[1] * (1 - 0.001), phi[1] * 0.001, phi[3]]
        else:
            prob = [0, 0, phi[2], phi[3]]
        sprime = np.random.choice(range(1,5), size = 1, p = prob)[0]
        return self.state_space[sprime] # return a string
    
    def next_state(self, phi):
        next_state = self.update_state(phi, self.h)
        self.add_state(next_state)
        return next_state
    
    def generate_reward(self, phi):
        reward = np.dot(phi, self.theta[self.h])
        self.R.append(reward)
        return reward
    
    def step(self, a):
        self.A.append(a)
        phi = self.phi(self.current_state, a)
        phi_a = [self.phi(self.current_state, a) for a in self.action_space]
        self.feature.append(phi)
        self.feature_a.append(phi_a)
        self.generate_reward(phi)
        self.next_state(phi)
        self.h += 1
        

class LinearMDP_test():
    """Perturbed evironment"""
    def __init__(self,  nominal_MDP, q, seed=1):
        np.random.seed(seed)
        self.state_space = nominal_MDP.state_space
        self.action_space = nominal_MDP.action_space
        self.initial_state = 'x1'
        self.theta = nominal_MDP.theta
        self.delta = nominal_MDP.delta
        self.Xi = nominal_MDP.Xi
        self.H = 3
        self.q = q

    def reset(self):
        self.S = [self.initial_state] # save the feature trajectory
        self.A = [] # save the action history
        self.R = [] # save the reward history
        self.h = 0 # reset the step to 0
        self.feature = [] # save the feature trajectory
        self.feature_a = [] # save the full feature trajectory
        self.current_state = self.initial_state # reset the current  state to initial state

    def phi(self, current_state, A):
        if current_state == 'x1':
            phi = np.array([1 - self.delta - self.Xi @ A, 0, 0, self.delta + self.Xi @ A])
        elif current_state == 'x2':
            phi = np.array([0, 1 - self.delta - self.Xi @ A, 0, self.delta + self.Xi @ A])
        elif current_state == 'x3':
            phi = np.array([0, 0, 1 - self.delta - self.Xi @ A, self.delta + self.Xi @ A])
        elif current_state == 'x4':
            phi = np.array([0,0,1,0])
        else:
            phi = np.array([0,0,0,1])
        return phi
    
    def add_state(self, s): 
        self.S.append(s)
        self.current_state = s

    def update_state(self, phi, h):
        # calculate the transition probability  --- perturbed
        if  h == 0:
            prob = [phi[0], 0, self.q * phi[3], (1 - self.q) * phi[3]]
        elif h == 1:
            if phi[2] == 1:
                prob = [0, 0, 1, 0]
            elif phi[3] == 1:
                prob = [0, 0, 0, 1]
            else:
                prob = [0, phi[1] * (1 - 0.001), phi[1] * 0.001, phi[3]]
        else:
            prob =  [0, 0, phi[2], phi[3]]
        sprime = np.random.choice(range(1,5), size = 1, p = prob)[0]
        return self.state_space[sprime] # return a string
    
    def next_state(self, phi):
        next_state = self.update_state(phi, self.h)
        self.add_state(next_state)
        return next_state
    
    def generate_reward(self, phi):
        reward = np.dot(phi, self.theta[self.h])
        self.R.append(reward)
        return reward

    def step(self, a):
        self.A.append(a)
        phi = self.phi(self.current_state, a)
        phi_a = [self.phi(self.current_state, a) for a in self.action_space]
        self.feature.append(phi)
        self.feature_a.append(phi_a)
        self.generate_reward(phi)
        self.next_state(phi)
        self.h += 1

In [66]:
class LSVI_LCB():
    def __init__(self, A, beta, H, lam, dataset, fail_state=None):
        self.lam = lam
        self.H = H
        self.action_space = A
        self.beta = beta
        self.w = [np.zeros(4) for _ in range(self.H)]
        self.fail_state = fail_state
        self.Lambda = [self.lam * np.diag(np.ones(4)) for _ in range(self.H)]
        self.dataset = dataset
    
    def get_action(self, phi_a, h):
        Q_h = [self.get_Q_func(phi_a[idx], h) for idx in range (len(self.action_space))]
        return self.action_space[np.argmax(Q_h)]
    
    def get_Q_func(self, phi, h, s_f=False):
        if s_f == True:
            return 0
        else:
            Lambda_h_inverse = np.linalg.inv(self.Lambda[h])
            Q_h = np.min([(self.w[h] @ phi - self.beta * np.sqrt(phi @ Lambda_h_inverse @ phi)), self.H])
            return Q_h
    
    def update_Q(self):
        # Backward induction
        self.w = [None for _ in range(self.H)] #initialize weights w
        for h in range(self.H-1, -1, -1):
            # calculate Lambda_h
            for n in range(self.dataset['k']):
                feature_temp = self.dataset['phi'][n][h]
                self.Lambda[h] += np.outer(feature_temp, feature_temp)
            #  calculate w_h
            Lambda_h_inverse = np.linalg.inv(self.Lambda[h])
            w_h = np.zeros(4)
            if h == self.H - 1:
                for tau in range(self.dataset['k']):
                    phi_tau_h = self.dataset['phi'][tau][h]
                    r_tau_h = self.dataset['r'][tau][h]
                    w_h += Lambda_h_inverse @ (phi_tau_h * r_tau_h)
            else:
                for tau in range(self.dataset['k']):
                    phi_tau_h = self.dataset['phi'][tau][h]
                    phi_tau_h_plus_one = self.dataset['phi_a'][tau][h+1]
                    r_tau_h = self.dataset['r'][tau][h]
                    s_f_h_plus_one = (self.dataset['state'][tau][h+1] == self.fail_state)
                    Q_tau_h_plus_one = [self.get_Q_func(phi_tau_h_plus_one[idx], h + 1, s_f_h_plus_one)
                                    for idx in range(len(self.action_space))]
                    V_tau_h_plus_one = np.max(Q_tau_h_plus_one)
                    w_h += Lambda_h_inverse @ (phi_tau_h * (r_tau_h + V_tau_h_plus_one))
            self.w[h] = w_h

class DR_LSVI_LCB():
    def __init__(self, A, beta, H, lam, dataset, Rho, theta, fail_state=None):
        self.lam = lam
        self.H = H
        self.action_space = A
        self.beta = beta
        self.w = [np.zeros(4) for _ in range(self.H)]
        self.Lambda = [self.lam * np.diag(np.ones(4)) for _ in range(self.H)]
        self.Rho = Rho
        self.theta = theta
        self.fail_state = fail_state
        self.dataset = dataset

    def get_action(self, phi_a, h):
        Q_h = [self.get_Q_func(phi_a[idx], h) for idx in range(len(self.action_space))]
        return self.action_space[np.argmax(Q_h)]

    def get_Q_func(self, phi, h, s_f=False):
        if s_f == True:
            return 0
        else:
            Lambda_h_inverse = np.linalg.inv(self.Lambda[h])
            penalty = self.beta * np.sqrt(phi @ np.diag(np.diagonal(Lambda_h_inverse)) @ phi)
            Q_h = np.min([(self.w[h] @ phi - penalty), self.H - h])
            return Q_h

    def get_nu_h(self, h, rho):
        Lambda_h_inverse = np.linalg.inv(self.Lambda[h])
        nu_h = np.zeros(4)
        Phi_h = np.zeros((0,4))
        V_h_plus_one = np.zeros(0)
        for tau in range(self.dataset['k']):
            phi_tau_h = self.dataset['phi'][tau][h]
            Phi_h = np.vstack((Phi_h, phi_tau_h))
            phi_tau_h_plus_one = self.dataset['phi_a'][tau][h+1]
            s_f_h_plus_one = (self.dataset['state'][tau][h+1] == self.fail_state)
            Q_tau_h_plus_one = [self.get_Q_func(phi_tau_h_plus_one[idx], h+1, s_f_h_plus_one)
                                for idx in range(len(self.action_space))]
            V_tau_h_plus_one = np.max(Q_tau_h_plus_one)
            V_h_plus_one = np.hstack((V_h_plus_one, V_tau_h_plus_one))
        for i in range(4):
            def z_alpha_i(alpha):
                # compact formular for z
                z = Lambda_h_inverse @ Phi_h.T @ np.minimum(V_h_plus_one, alpha)
                return -z[i] + rho[i] * alpha
            result = minimize(z_alpha_i, self.H/2, method='Nelder-Mead', bounds=[(0, self.H)])
            nu_h[i] = - result.fun
            return nu_h

    def update_Q(self):
        # Backward induction
        self.w = [None for _ in range(self.H)] # innitialize weights w
        for h in range(self.H-1, -1, -1):
            # calculate Lambda_h
            for n in range(self.dataset['k']):
                feature_temp = self.dataset['phi'][n][h]
                self.Lambda[h] += np.outer(feature_temp, feature_temp)
            # update w_h
            w_h = np.zeros(4)
            nu_h = np.zeros(4)
            if h == self.H - 1:
                w_h = self.theta[h]
            else:
                nu_h = self.get_nu_h(h, rho=self.Rho[h])
                w_h = self.theta[h] + nu_h
            self.w[h] = w_h

class VA_DR_LSVI_LCB():
    def __init__(self, pre_agent, A, beta, H, lam, dataset, Rho, theta, fail_state=None):
        self.lam = lam
        self.H = H
        self.action_space = A
        self.beta = beta
        self.w = [np.zeros(4) for _ in range(self.H)]
        self.Lambda = [self.lam * np.diag(np.ones(4)) for _ in range(self.H)]
        self.Rho = Rho
        self.theta = theta
        self.fail_state = fail_state
        self.dataset = dataset
        self.pre_agent = pre_agent
        self.variance = {}

    def get_variance_coefficient(self, h):
        Lambda_h_inverse = np.linalg.inv(self.Lambda[h])
        z1_h = np.zeros(4)
        z2_h = np.zeros(4)
        Phi_h = np.zeros((0,4))
        V_h_plus_one = np.zeros(0)
        for tau in range(self.dataset['k']):
            phi_tau_h = self.dataset['phi'][tau][h]
            Phi_h = np.vstack((Phi_h, phi_tau_h))
            phi_tau_h_plus_one = self.dataset['phi_a'][tau][h+1]
            s_f_h_plus_one = (self.dataset['state'][tau][h+1] == self.fail_state)
            Q_tau_h_plus_one = [self.pre_agent.get_Q_func(phi_tau_h_plus_one[idx], h+1, s_f_h_plus_one)
                                for idx in range(len(self.action_space))]
            V_tau_h_plus_one = np.max(Q_tau_h_plus_one)
            V_h_plus_one = np.hstack((V_h_plus_one, V_tau_h_plus_one))
        
        z1_h = Lambda_h_inverse @ Phi_h.T @ V_h_plus_one
        z2_h = Lambda_h_inverse @ Phi_h.T @ V_h_plus_one**2
        return z1_h, z2_h
    
    def estimated_variance(self, phi, z1_h, z2_h):
        second_order_term = np.min([np.max([0, np.dot(phi, z2_h)]), self.H**2])
        first_order_term = np.min([np.max([0, np.dot(phi, z1_h)]), self.H])
        sigma_square = np.max([1, second_order_term - first_order_term**2])
        return sigma_square
    
    def get_action(self, phi_a, h):
        Q_h = [self.get_Q_func(phi_a[idx], h) for idx in range(len(self.action_space))]
        return self.action_space[np.argmax(Q_h)]
    
    def get_Q_func(self, phi, h, s_f=False):
        if s_f == True:
            return 0
        else:
            Lambda_h_inverse = np.linalg.inv(self.Lambda[h])
            penalty = self.beta * np.sqrt(phi @ np.diag(np.diagonal(Lambda_h_inverse)) @ phi)
            Q_h = np.min([(self.w[h] @ phi - penalty), self.H - h])
            return Q_h

    def get_nu_h(self, h, rho, variance):
        Lambda_h_inverse = np.linalg.inv(self.Lambda[h])
        nu_h = np.zeros(4)
        Phi_h = np.zeros((0, 4))
        V_h_plus_one = np.zeros(0)
        for tau in range(self.dataset['k']):
            phi_tau_h = self.dataset['phi'][tau][h]
            Phi_h = np.vstack((Phi_h, phi_tau_h))
            phi_tau_h_plus_one = self.dataset['phi_a'][tau][h+1]
            s_f_h_plus_one = (self.dataset['state'][tau][h+1] == self.fail_state)
            Q_tau_h_plus_one = [self.get_Q_func(phi_tau_h_plus_one[idx], h+1, s_f_h_plus_one)
                                for idx in range(len(self.action_space))]
            V_tau_h_plus_one = np.max(Q_tau_h_plus_one)
            V_h_plus_one = np.hstack((V_h_plus_one, V_tau_h_plus_one))
        for i in range(4):
            def z_alpha_i(alpha):
                # compact formular for z
                z = Lambda_h_inverse @ Phi_h.T @ (np.minimum(V_h_plus_one, alpha) / variance)
                return -z[i] + rho[i] * alpha
            result = minimize(z_alpha_i, self.H/2, method='Nelder-Mead', bounds=[(0, self.H)])
            nu_h[i] = - result.fun
            return nu_h
        
    def update_Q(self):
        # Backward induction
        self.w = [None for _ in range(self.H)] # initialize weights w
        for h in range(self.H-1, -1, -1):
            # calculate Lambda_h
            if h == self.H-1:
                for n in range(self.dataset['k']):
                    feature_temp = self.dataset['phi'][n][h]
                    self.Lambda[h] += np.outer(feature_temp, feature_temp)
            else:
                z1_h, z2_h = self.get_variance_coefficient(h)
                self.variance[str(h)] = np.zeros(self.dataset['k'])
                for n in range(self.dataset['k']):
                    feature_temp = self.dataset['phi'][n][h]
                    variance_temp = self.estimated_variance(feature_temp, z1_h, z2_h)
                    self.variance[str(h)][n] = variance_temp
                    self.Lambda[h] += np.outer(feature_temp, feature_temp) / variance_temp
            # update w_h
            w_h = np.zeros(4)
            nu_h = np.zeros(4)
            if h == self.H - 1:
                w_h = self.theta[h]
            else:
                variance = self.variance[str(h)]
                nu_h = self.get_nu_h(h, rho = self.Rho[h], variance = variance)
                w_h = self.theta[h] + nu_h
            self.w[h] = w_h


        


In [67]:

def Offline_Dataset_Collection(sample_size, env, seed=1):
    np.random.seed(seed)
    history = {'k': 0, 'phi':[], 'r':[], 'state':[], 'phi_a':[]}    
    epoch = sample_size
    for t in range(epoch):
        env.reset()
        for h in range(env.H):
            random_action_index = np.random.choice(range(0,len(env.action_space)), size = 1)[0]
            action = env.action_space[random_action_index]
            env.step(action)
        # log the trajectory
        history['phi_a'].append(env.feature_a)
        history['phi'].append(env.feature)
        history['r'].append(env.R)
        history['state'].append(env.S)
        history['k'] += 1
    return history

In [68]:
def train_once(dataset, action_space, beta, H, lam, fail_state):
    agent = LSVI_LCB(A=action_space, beta=beta, H=H, lam=lam, dataset=dataset, fail_state=fail_state)
    agent.update_Q()
    return agent

def train_once_DR(dataset, action_space, beta, H, lam, Rho, theta, fail_state):
    agent = DR_LSVI_LCB(A=action_space, beta=beta, H=H, lam=lam, dataset=dataset, Rho=Rho, theta=theta, fail_state=fail_state)
    agent.update_Q()
    return agent

def train_once_DR_VA(dataset, action_space, beta, H, lam, Rho, theta, pre_agent, fail_state):
    agent = VA_DR_LSVI_LCB(pre_agent=pre_agent, A=action_space, beta=beta, H=H, lam=lam, dataset=dataset, Rho=Rho, theta=theta, fail_state=fail_state)
    agent.update_Q()
    return agent


In [88]:
T1 = 100
H = 3
rho = 0.3
beta = 1
lam = 0.1
actions = list(product([-1, 1], repeat=4))
action_space = [np.array(action) for action in actions]
delta = 0.3
xi_norm = 0.1
Rho = [[0,0,0,rho], [0,0,0,0]]
fail_state = 'x4'
replication = 20
agent_dic = {}
DR_agent_dic = {}
VA_DR_agent_dic = {}
env = LinearMDP_train(action_space, delta, xi_norm)

for rep in range(replication):
    Offline_Dataset = Offline_Dataset_Collection(T1, env, seed=rep)
    agent = train_once(dataset=Offline_Dataset, action_space=action_space, beta=beta, H=H, lam=lam, fail_state=fail_state)
    DR_agent = train_once_DR(dataset=Offline_Dataset, action_space=action_space, beta=beta, H=H, lam=lam, Rho=Rho, theta=env.theta, fail_state=fail_state)
    VA_DR_agent = train_once_DR_VA(pre_agent=DR_agent, dataset=Offline_Dataset, action_space=action_space, beta=beta, H=H, lam=lam, Rho=Rho, theta=env.theta, fail_state=fail_state)
    agent_dic[str(rep)] = agent
    DR_agent_dic[str(rep)] = DR_agent
    VA_DR_agent_dic[str(rep)] = VA_DR_agent


ValueError: probabilities are not non-negative

In [89]:
Perturbation = [x / 20 for x in range(21)]
T2 = 100
R_LSVI_LCB = []
R_DR_LSVI_LCB = []
R_VA_DR_LSVI_LCB = []
env = LinearMDP_train(action_space, delta, xi_norm)
for q in Perturbation:
    REWARD = 0
    REWARD_DR = 0
    REWARD_DR_VA = 0
    for rep in range(replication):
        reward = 0
        reward_DR = 0
        reward_DR_VA = 0
        env_test = LinearMDP_test(env, q=q, seed=rep)
        env_test_DR = LinearMDP_test(env, q=q, seed=rep)
        env_test_DR_VA = LinearMDP_test(env, q=q, seed=rep)
        agent = agent_dic[str(rep)]
        DR_agent = DR_agent_dic[str(rep)]
        VA_DR_agent = VA_DR_agent_dic[str(rep)]

        for t in range(T2):
            env_test.reset()
            env_test_DR.reset()
            env_test_DR_VA.reset()
            for h in range(H):
                # PEVI
                current_state = env_test.current_state
                phi_a = [env_test.phi(current_state, a) for a in action_space]
                action = agent.get_action(phi_a, h)
                env_test.step(action)
                # DRPVI
                current_state_DR = env_test_DR.current_state
                phi_DR_a = [env_test_DR.phi(current_state_DR, a) for a in action_space]
                action_DR = DR_agent.get_action(phi_DR_a, h)
                env_test_DR.step(action_DR)
                
                # VA-DRPVI
                current_state_DR_VA = env_test_DR_VA.current_state
                phi_DR_VA_a = [env_test_DR_VA.phi(current_state_DR_VA, a) for a in action_space]
                action_DR_VA = VA_DR_agent.get_action(phi_DR_VA_a, h)
                env_test_DR_VA.step(action_DR_VA)
                
                

            reward += np.sum(env_test.R) / T2
            reward_DR += np.sum(env_test_DR.R) / T2   
            reward_DR_VA += np.sum(env_test_DR_VA.R) / T2 

        REWARD += reward / replication
        REWARD_DR += reward_DR / replication 
        REWARD_DR_VA += reward_DR_VA / replication 
    
    R_LSVI_LCB.append(REWARD)
    R_DR_LSVI_LCB.append(REWARD_DR)
    R_VA_DR_LSVI_LCB.append(REWARD_DR_VA)

plt.plot(Perturbation, R_LSVI_LCB, label = 'PEVI')
plt.plot(Perturbation, R_DR_LSVI_LCB, label = 'DRPVI')
plt.plot(Perturbation, R_VA_DR_LSVI_LCB, label = 'VA-DRPVI')
plt.legend(fontsize=16)
plt.xlabel('Perturbation', size=16)
plt.ylabel('Average reward', size=16)
plt.savefig(f'robustness_{delta}_{xi_norm}_{rho}.pdf', dpi=1000, bbox_inches='tight', pad_inches=0.0)



KeyError: '0'